In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [3]:
%%sql
SELECT * FROM playlist;

Done.


playlist_id,name
1,Music
2,Movies
3,TV Shows
4,Audiobooks
5,90’s Music
6,Audiobooks
7,Movies
8,Music
9,Music Videos
10,TV Shows


## Question: Number of tracks grouped by genres in USA

Logic from Schema 
1. Genre to track (Grouped by genre) -- 
2. Customer to country to invoice to track_id


In [29]:
%%sql
WITH us_inv AS (SELECT billing_country,
                       invoice_id
    FROM invoice
    WHERE billing_country = 'USA'),

us_tracks AS(
    SELECT il.track_id track_id,
           ius.billing_country
    FROM us_inv ius       
    INNER JOIN invoice_line il ON il.invoice_id = ius.invoice_id),

genre_track AS (SELECT g.name genre,
                       t.track_id track_id
    FROM genre g
    LEFT JOIN track t ON t.genre_id = g.genre_id
),

total_us_tracks AS(
   SELECT COUNT(ut.track_id) total_tracks
   FROM us_tracks ut)


SELECT gt.genre,
       COUNT(ut.track_id) N_of_tracks,
       COUNT(ut.track_id)*100/(SELECT CAST(total_tracks AS FLOAT) FROM total_us_tracks)  
FROM us_tracks ut 
INNER JOIN genre_track gt ON ut.track_id = gt.track_id
GROUP BY 1;


Done.


genre,N_of_tracks,COUNT(ut.track_id)*100/(SELECT total_tracks FROM total_us_tracks)
Alternative,35,3
Alternative & Punk,130,12
Blues,36,3
Classical,4,0
Easy Listening,13,1
Electronica/Dance,5,0
Heavy Metal,3,0
Hip Hop/Rap,20,1
Jazz,14,1
Latin,22,2


In [ ]:
SELECT gt.genre,
       COUNT(ut.track_id)
FROM genre_track gt INNER JOIN us_tracks ut ON gt.track_id = ut.track_id
GROUP BY 1
ORDER BY 1;


genre_track AS (SELECT g.name, genre
                            t.track_id track_id
    FROM genre g
    LEFT JOIN track t ON t.genre_id = g.genre_id
)




